In [1]:
from bs4 import BeautifulSoup as bfs 
from urllib.request import urlopen , Request
import pandas as pd
import numpy as np
import requests as rq
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located

In [4]:
option=webdriver.ChromeOptions()
option.add_argument("headless")

In [5]:
url='https://racing.hkjc.com/racing/information/Chinese/Racing/ResultsAll.aspx?RaceDate=2021/03/21'
driver = webdriver.Chrome(options=option)
driver.get(url)
time.sleep(3)
a=driver.page_source

In [6]:
page_soup=bfs(a,"html.parser")

In [9]:
contents=page_soup.findAll("option")

In [10]:
dates=[i.text.split("/")[2]+"/"+i.text.split("/")[1]+"/"+i.text.split("/")[0] for i in contents]

In [11]:
print("the numbers of dates is: ",len(dates))

the numbers of dates is:  179


In [30]:
N=int(input("How many dates you want:"))

How many dates you want:4


In [31]:
a=[]
Date,race,Pla,HNo,Horse,Jockey,Trainer,ActualWt,Dr=[],[],[],[],[],[],[],[],[]
print('that may take a lot of time')
for elt in dates[:N]:
    Url="https://racing.hkjc.com/racing/information/Chinese/Racing/ResultsAll.aspx?RaceDate="+elt
    driver.get(Url)
    time.sleep(3)
    a.append(bfs(driver.page_source,"html.parser")) 

that may take a lot of time


In [32]:
Date,race,Pla,HNo,Horse,Jockey,Trainer,ActualWt,Dr=[],[],[],[],[],[],[],[],[]
i=0

for elt in a:
    contents=elt.findAll("table",{"class":"f_fs13 f_tac result"})
    for j in range(len(contents)):
        cn=contents[j].findAll("tr")
        race+=[ "Race"+str(j+1) for r in range(len(cn)-1)]
        Date+=[dates[i] for j in range(len(cn)-1)]
        for l in range(1,len(cn)):
            cr=cn[l].findAll("td")
            Pla+=[cr[0].text]
            HNo+=[cr[1].text]
            Horse+=[cr[2].text]
            Jockey+=[cr[3].text]
            Trainer+=[cr[4].text]
            ActualWt+=[cr[5].text]
            Dr+=[cr[6].text] 
    i+=1

In [33]:
df1=pd.DataFrame({"Date":Date,"Race":race,"名次":Pla,'馬號':HNo, '馬名':Horse, 
                  '騎師':Jockey, '練馬師':Trainer, '實際 負磅':ActualWt,'檔位':Dr})

In [34]:
df1

,Date,Race,名次,馬號,馬名,騎師,練馬師,實際 負磅,檔位
0,2021/03/28,Race1,1,4,生生勝勝,莫雷拉,姚本輝,130,9
1,2021/03/28,Race1,2,9,合金之星,潘頓,容天鵬,121,1
2,2021/03/28,Race1,3,8,種福者,郭能,文家良,125,7
3,2021/03/28,Race1,4,10,坪山新星,史卓豐,蘇保羅,120,2
4,2021/03/28,Race2,1,7,龍戰士,梁家俊,蘇保羅,119,6
...,...,...,...,...,...,...,...,...,...
112,2021/03/21,Race9,4,9,鑽飛龍,潘頓,容天鵬,122,8
113,2021/03/21,Race10,1,6,精神威,梁家俊,羅富全,126,12
114,2021/03/21,Race10,2,1,美麗笑容,潘頓,告東尼,131,13
115,2021/03/21,Race10,3,14,喜駿駒,莫雷拉,蔡約翰,118,3


In [35]:
df1.shape

(117, 9)

In [36]:
df2=df1[df1["名次"]=="1"].groupby("騎師").count().iloc[:,[0,1]]
df2.columns=["騎師","total"]
df3=df1[df1["名次"]=="2"].groupby("騎師").count().iloc[:,[0,1]]
df3.columns=["騎師","total"]
df4=df1[df1["名次"]=="3"].groupby("騎師").count().iloc[:,[0,1]]
df4.columns=["騎師","total"]

In [37]:
DF=df1[(df1["名次"]=="1") | (df1["名次"]=="2") ]
DF2=df1[(df1["名次"]=="1") | (df1["名次"]=="3") ]
DF3=df1[(df1["名次"]=="2") | (df1["名次"]=="3") ]

In [38]:
list=[DF.iloc[i,5]+" and "+DF.iloc[i+1,5] for i in range(0,DF.shape[0],2) if DF.iloc[i,5]!=DF.iloc[i+1,5] ]
list2=[DF2.iloc[i,5]+" and "+DF2.iloc[i+1,5] for i in range(0,DF2.shape[0],2) if DF2.iloc[i,5]!=DF2.iloc[i+1,5] ]
list3=[DF3.iloc[i,5]+" and "+DF3.iloc[i+1,5] for i in range(0,DF3.shape[0],2) if DF3.iloc[i,5]!=DF3.iloc[i+1,5] ]

list4=[DF.iloc[i,6]+" and "+DF.iloc[i+1,6] for i in range(0,DF.shape[0],2)  ]
list5=[DF2.iloc[i,6]+" and "+DF2.iloc[i+1,6] for i in range(0,DF2.shape[0],2) ]
list6=[DF3.iloc[i,6]+" and "+DF3.iloc[i+1,6] for i in range(0,DF3.shape[0],2) ]

In [39]:
first_second=pd.DataFrame({"first_and_second":list,"number":[1 for i in range(len(list))]})
first_third=pd.DataFrame({"first_and_third":list2,"number":[1 for i in range(len(list2))]})
second_third=pd.DataFrame({"second_and_third":list3,"number":[1 for i in range(len(list3))]})

trainer1_2=pd.DataFrame({"trainer1_2":list4,"number":[1 for i in range(len(list4))]})
trainer1_3=pd.DataFrame({"trainer1_3":list5,"number":[1 for i in range(len(list5))]})
trainer2_3=pd.DataFrame({"trainer2_3":list6,"number":[1 for i in range(len(list6))]})

In [41]:
first_second=first_second.groupby("first_and_second").count()
first_third=first_third.groupby("first_and_third").count()
second_third=second_third.groupby("second_and_third").count()

trainer1_2=trainer1_2.groupby("trainer1_2").count()
trainer1_3=trainer1_3.groupby("trainer1_3").count()
trainer2_3=trainer2_3.groupby("trainer2_3").count()

In [42]:
y=pd.ExcelWriter("All_data_china.xlsx", engine='xlsxwriter')
df1.to_excel(y, sheet_name='total')
df2.loc[:,["total"]].to_excel(y, sheet_name='firstplace_by_jockey')
df3.loc[:,["total"]].to_excel(y, sheet_name='second_by_Jockey')
df4.loc[:,["total"]].to_excel(y, sheet_name='third_by_Jockey')
first_second.to_excel(y,sheet_name="combination_1_2")
first_third.to_excel(y,sheet_name="combination_1_3")
second_third.to_excel(y,sheet_name="combination_2_3")
trainer1_2.to_excel(y,sheet_name="trainer1_2")
trainer1_3.to_excel(y,sheet_name="trainer1_3")
trainer2_3.to_excel(y,sheet_name="trainer2_3")
y.save()